# Plots


Notebook to test different kinds of plots to visualize the results of multiple experiments.

In [ ]:
# Import modules
import pandas as pd
import graphviz
import typing
import matplotlib.pyplot as plt
import numpy as np
import itertools

from collections import defaultdict
from typing import Dict


from lib.metrics import *
from lib.message import *
from lib.analysis import PubSubAnalyzer
from lib.experiment import PubSubExperiment, PubSubExperimentResults


In [ ]:
# Load all the different experiments

# Results for all protocols and configurations
results = {
    # "quick": {
    #     "gossipsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/gossipsub-quick.json"
    #     ),
    #     "plumtree": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/plumtree-quick.json"
    #     ),
    #     "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-quick.json"
    #     ),
    #     "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-rfac2-quick.json"
    #     ),
    # },
    "config1": {
        "gossipsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/gossipsub-config1.json"
        ),
        "plumtree": PubSubAnalyzer.from_experiment_results_file(
            "experiments/plumtree-config1.json"
        ),
        "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-config1.json"
        ),
        "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-rfac2-config1.json"
        ),
    },
    "config2": {
        "gossipsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/gossipsub-config2.json"
        ),
        "plumtree": PubSubAnalyzer.from_experiment_results_file(
            "experiments/plumtree-config2.json"
        ),
        "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-config2.json"
        ),
        "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-rfac2-config2.json"
        ),
    },
    "config3": {
        "gossipsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/gossipsub-config3.json"
        ),
        "plumtree": PubSubAnalyzer.from_experiment_results_file(
            "experiments/plumtree-config3.json"
        ),
        "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-config3.json"
        ),
        "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
            "experiments/kadpubsub-rfac2-config3.json"
        ),
    },
    # "config4": {
    #     "gossipsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/gossipsub-config4.json"
    #     ),
    #     "plumtree": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/plumtree-config4.json"
    #     ),
    #     "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-config4.json"
    #     ),
    #     "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-rfac2-config4.json"
    #     ),
    # },
    # "config5": {
    #     "gossipsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/gossipsub-config5.json"
    #     ),
    #     "plumtree": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/plumtree-config5.json"
    #     ),
    #     "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-config5.json"
    #     ),
    #     "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-rfac2-config5.json"
    #     ),
    # },
    # "config6": {
    #     "gossipsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/gossipsub-config6.json"
    #     ),
    #     "plumtree": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/plumtree-config6.json"
    #     ),
    #     "kadpubsub": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-config6.json"
    #     ),
    #     "kadpubsub-rfac2": PubSubAnalyzer.from_experiment_results_file(
    #         "experiments/kadpubsub-rfac2-config6.json"
    #     ),
    # },
}


In [ ]:
# Make sure the configurations are the same and configure plot dpi

plt.rcParams["figure.dpi"] = 600


def _check_configs_have_equivalent_experiment_setups():
    config_experiments = [
        [(name, v.experiment) for name, v in config.items()]
        for config in results.values()
    ]
    for (n1, e1), (n2, e2) in itertools.combinations(config_experiments, 2):
        assert e1.is_equivalent(e2, compare_protocol_parameters=n1 == n2)


#_check_configs_have_equivalent_experiment_setups()


In [ ]:
# Define function to check if we are running in a notebook

# https://stackoverflow.com/questions/15411967/how-can-i-check-if-code-is-executed-in-the-ipython-notebook
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == "ZMQInteractiveShell":
            return True  # Jupyter notebook or qtconsole
        elif shell == "TerminalInteractiveShell":
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False  # Probably standard Python interpreter


In [ ]:
configs = list(results.keys())
protocols = list(set([k for config in results.values() for k in config.keys()]))
config_experiments = {
    config_name: [v.experiment for v in config.values()][0]
    for config_name, config in results.items()
}
config_descriptions = {
    k: f"{v.number_nodes} nodes\n{v.payload_size} bytes\n{v.broadcast_rate} messages/s"
    for k, v in config_experiments.items()
}

### Bar plot configuration
# Relevant: https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
bar_plot_x = np.arange(len(configs))
bar_plot_group_width = 0.5
bar_plot_bar_width = bar_plot_group_width / len(protocols)
bar_plot_bar_offset = np.array(
    [bar_plot_group_width / 2 - i * bar_plot_bar_width for i in range(len(protocols))]
)
# Descriptions that show at the bottom of each bar group
bar_plot_labels = [config_descriptions[c] for c in configs]


In [ ]:
# Plot reliability as a box plot

reliability_per_protocol = {
    protocol: [
        results[c][protocol].reliability()["reliability"].mean() for c in configs
    ]
    for protocol in protocols
}

fig, ax = plt.subplots()

# Bars for each protocol and configuration
bars = [
    ax.bar(
        bar_plot_x + bar_plot_bar_offset[i],
        reliability_per_protocol[protocol],
        bar_plot_bar_width,
        label=protocol,
    )
    for i, protocol in enumerate(protocols)
]

ax.set_title("Publish/Subscribe Reliability")
ax.set_xticks(bar_plot_x, bar_plot_labels)
ax.set_ylabel("Reliability")
ax.set_ylim(0, 1.2)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

for bar in bars:
    ax.bar_label(bar, padding=3, fmt="%.3f")

if is_notebook():
    plt.show()
else:
    plt.savefig("plots/reliability.png")


In [ ]:
# Plot publish latency as a box plot

publish_latencies_per_protocol = {
    protocol: [
        results[c][protocol].publish_latency()["latency"].mean() for c in configs
    ]
    for protocol in protocols
}

fig, ax = plt.subplots()
bars = [
    ax.bar(
        bar_plot_x + bar_plot_bar_offset[i],
        publish_latencies_per_protocol[protocol],
        bar_plot_bar_width,
        label=protocol,
    )
    for i, protocol in enumerate(protocols)
]

ax.set_title("Publish Latency")
ax.set_xticks(bar_plot_x, bar_plot_labels)
ax.set_ylabel("Latency (hops)")
ax.set_ylim(
    0, max([l for ls in publish_latencies_per_protocol.values() for l in ls]) + 2
)
ax.legend()

for bar in bars:
    ax.bar_label(bar, padding=3, fmt="%.3f")

if is_notebook():
    plt.show()
else:
    plt.savefig("plots/publish-latency.png")


In [ ]:
# Plot redundancy

redundancy_per_protocol = {
    protocol: [results[c][protocol].redundancy() for c in configs]
    for protocol in protocols
}

fig, ax = plt.subplots()
bars = [
    ax.bar(
        bar_plot_x + bar_plot_bar_offset[i],
        redundancy_per_protocol[protocol],
        bar_plot_bar_width,
        label=protocol,
    )
    for i, protocol in enumerate(protocols)
]

ax.set_title("Redundancy")
ax.set_xticks(bar_plot_x, bar_plot_labels)
ax.set_ylabel("Redundancy")
ax.set_ylim(0, 1.2)
ax.legend()

for bar in bars:
    ax.bar_label(bar, padding=3, fmt="%.3f")

if is_notebook():
    plt.show()
else:
    plt.savefig("plots/redundancy.png")


In [ ]:
# Network usefullness

network_usefulness_per_protocol = {
    protocol: [
        results[c][protocol].network_usage_usefullness_fraction() for c in configs
    ]
    for protocol in protocols
}

fig, ax = plt.subplots()
bars = [
    ax.bar(
        bar_plot_x + bar_plot_bar_offset[i],
        network_usefulness_per_protocol[protocol],
        bar_plot_bar_width,
        label=protocol,
    )
    for i, protocol in enumerate(protocols)
]

ax.set_title("Network Usefulness")
ax.set_xticks(bar_plot_x, bar_plot_labels)
ax.set_ylabel("Usefulness")
ax.set_ylim(0, 1.2)
ax.legend()

for bar in bars:
    ax.bar_label(bar, padding=3, fmt="%.3f")

if is_notebook():
    plt.show()
else:
    plt.savefig("plots/network-usefulness.png")


In [ ]:
# Network usage

network_usage_per_protocol = {
    protocol: [results[c][protocol].network_usage() / (1024**3) for c in configs]
    for protocol in protocols
}

fig, ax = plt.subplots()
bars = [
    ax.bar(
        bar_plot_x + bar_plot_bar_offset[i],
        network_usage_per_protocol[protocol],
        bar_plot_bar_width,
        label=protocol,
    )
    for i, protocol in enumerate(protocols)
]

ax.set_title("Network Usage")
ax.set_xticks(bar_plot_x, bar_plot_labels)
ax.set_yscale("log")
ax.set_ylabel("GB")
ax.legend()

for bar in bars:
    ax.bar_label(bar, padding=3, fmt="%.3f")

if is_notebook():
    plt.show()
else:
    plt.savefig("plots/network-usage.png")


In [ ]:



# class PubSubPlotter:
#     def __init__(self, experiments: Dict[str, PubSubAnalyzer]):
#         self._experiments = experiments

#     def redundancy(self):
#         fig, ax = plt.subplots()
#         for i, (name, exp) in enumerate(self._experiments.items()):
#             exp_redundancy = exp.redundancy()
#             ax.bar(name, exp_redundancy, label="Redundant Messages", color="red")
#             ax.text(
#                 i,
#                 exp_redundancy / 2,
#                 "{0:.1f}%".format(exp_redundancy * 100),
#                 color="white",
#                 ha="center",
#                 va="center",
#                 fontweight="bold",
#             )
#             ax.bar(
#                 name,
#                 1 - exp_redundancy,
#                 bottom=exp_redundancy,
#                 label="Relevant Messages",
#                 color="green",
#             )
#             ax.text(
#                 i,
#                 exp_redundancy + (1 - exp_redundancy) / 2,
#                 "{0:.1f}%".format((1 - exp_redundancy) * 100),
#                 color="white",
#                 ha="center",
#                 va="center",
#                 fontweight="bold",
#             )
#         ax.set(xlabel="Configuration", ylabel="Redundancy", ylim=(0, 1.05))
#         fig.tight_layout()
#         return fig

#     def reliability(self):
#         reliabilities = pd.DataFrame(
#             {
#                 name: exp.reliability()["reliability"].resample("1s").mean()
#                 for name, exp in self._experiments.items()
#             }
#         )
#         return reliabilities.fillna(method="bfill").fillna(0).plot(title="Reliability")


# ploter = PubSubPlotter({"KadPubSub": exp, "GossipSub": exp2, "Plumtree": exp3})

# ploter.reliability().figure.savefig("reliability.pdf", dpi=600)
